In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

%matplotlib inline

# Load dataset

In [2]:
# read data
song = pd.read_json('/Users/kexinluo/Desktop/DS_take_home/8.Song/song.json')
song['time_played'] = pd.to_datetime(song['time_played'])
song['user_sign_up_date'] = pd.to_datetime(song['user_sign_up_date'])
song.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [3]:
song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4000 non-null   object        
 1   user_id            4000 non-null   int64         
 2   user_state         4000 non-null   object        
 3   user_sign_up_date  4000 non-null   datetime64[ns]
 4   song_played        4000 non-null   object        
 5   time_played        4000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


In [4]:
# check unique values
for name in song.columns:
    print('{0:15s} \t {1:5d}'.format(name, len(song[name].unique())))

id              	  4000
user_id         	   196
user_state      	    41
user_sign_up_date 	    20
song_played     	    97
time_played     	  3997


# Analysis

## Question 1

In [5]:
def unique_count(x):
    """ function to count the unique user_id """
    return len(np.unique(x))

In [6]:
# create temporary dataframe for unique user count per state
state_user_count = song.groupby('user_state')['user_id'].apply(unique_count).reset_index()
state_user_count = state_user_count.rename(columns={'user_id': 'user_count'})
state_user_count = state_user_count.sort_values(by='user_count', ascending=False)

In [7]:
# get the top 3 states
state_user_count.head(3)

,user_state,user_count
25,New York,23
4,California,21
35,Texas,15


In [8]:
# get the bottom 3 states
state_user_count.tail(9)

,user_state,user_count
24,New Mexico,1
9,Idaho,1
27,North Dakota,1
6,Connecticut,1
12,Iowa,1
32,Rhode Island,1
22,Nebraska,1
2,Arizona,1
13,Kansas,1


## Question 2

Here, we define the user engagement is measured throught the average plays per user in the given state.

In [9]:
# create temporary dataframe for play count per state
state_play_count = song.groupby('user_state')['id'].count().reset_index()
state_play_count = state_play_count.rename(columns={'id': 'play_count'})

# merge user count and play count
state_user_play = pd.merge(left=state_user_count, right=state_play_count, on='user_state')

# calculate average play
state_user_play['average_play'] = state_user_play['play_count'] / state_user_play['user_count']

# sort according to average_play
state_user_play = state_user_play.sort_values(by='average_play', ascending=False)


In [10]:
# get the top 3 states
state_user_play.head(3)

,user_state,user_count,play_count,average_play
38,Nebraska,1,36,36.000000
27,Alaska,2,58,29.000000
22,South Carolina,3,85,28.333333


In [11]:
# get the bottom 3 states
state_user_play.tail(3)[::-1]

,user_state,user_count,play_count,average_play
40,Kansas,1,8,8.0
30,Virginia,2,17,8.5
17,Minnesota,4,42,10.5


## Question 3

In [12]:
def find_first_user(df):
    """ function to find the first user """
    return df.iloc[df['user_sign_up_date'].argmin(), [1,3]]

In [13]:
song.groupby('user_state').apply(find_first_user).sort_values(by='user_sign_up_date')

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,19,2015-05-02


## Question 4

In [14]:
song.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


There should be a lot of different methods for this question. The simplest method is to choose the most popular songs. In addition, k-nearest neighbor (KNN) method can also be used. More advancely, collaborative filtering method can be implemented.

Here, I implement a simple version of collaborative filtering algorithm for song recommendation. More specifically, the similarity of two songs is calculate using the number of users whole listen them together.